<a href="https://colab.research.google.com/github/Computer-CGuy/BitcoinTPU/blob/main/Old_Songs_%7C_JaxEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [ ]:
import jax
import jax.numpy as np
import jax.numpy as jnp

In [ ]:
from jax import grad, jit, vmap
from jax import random

In [ ]:
from jax import device_put

## Main Code


In [ ]:
_h = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
def _rotr(x, y):
    return ((x >> y) | (x << (32 - y))) 
def _maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)
def _ch(x, y, z):
    return (x & y) ^ ((~x) & z)
def sigma0(x):
	return _rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22)
def sigma1(x):
	return _rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25)

In [ ]:
def body_fun(i,sampled):
    # print(val)
    # w,a,b,c,d,e,f,g,h,k,i = val
    # a,b,c,d,e,f,g,h
    # 0,1,2,3,4,5,6,7
    s0 = sigma0(sampled[0])
    s1 = sigma1(sampled[4])


    t1 = sampled[7] + s1 + _ch(sampled[4], sampled[5], sampled[6])
    # k[i]+w[i]
    t2 = s0 + _maj(sampled[0], sampled[1], sampled[2]) 

    jax.ops.index_update(sampled, jax.ops.index[1:], sampled[:-1])
    jax.ops.index_update(sampled, jax.ops.index[0], t1+t2)
    jax.ops.index_update(sampled, jax.ops.index[3], (sampled[3] + t1) )
    # sampled[0]=t1+t2
    # sampled[4]=(sampled[3] + t1) 
    # h = g
    # g = f
    # f = e
    # e = (sampled[3] + t1) 
    # d = c
    # c = b	
    # b = a
    # a = (t1 + t2)
    return sampled

In [ ]:
import numpy as npo

In [ ]:
key = random.PRNGKey(0)
import time
def callme(init_val):
    return jax.lax.fori_loop(0, 64, body_fun, init_val)#.block_until_ready()
callme = jit(callme)

In [ ]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [ ]:
# in_array_size = 1000
in_array_size = 10000
in_d = 1000
ls = [10,100,1000,10000,100000]

for in_d in ls:
    for in_array_size in ls:
        init_val = npo.random.randint(low=0,high=1000,size=(8,in_array_size,in_d),dtype=npo.int32)
        init_val = device_put(init_val)
        t0 = time.time()

        jax.vmap(callme,in_axes=1)(init_val).block_until_ready()
        t1 = time.time()
        print(in_array_size,in_d,"{:,}".format((in_array_size*in_d)/(t1-t0)))
        # print()

10 10 524.3922229543122
100 10 6,981.105444667847
1000 10 69,029.99641214866
10000 10 580,228.5608358891
100000 10 2,874,500.219992297
10 100 4,808.404582900563
100 100 63,943.79939506293
1000 100 521,222.5598200583
10000 100 2,748,546.5344873685
100000 100 2,306,965.4305880344
10 1000 50,152.38332105721
100 1000 566,295.8242422934
1000 1000 1,195,516.6529328083
10000 1000 2,771,840.406961798


In [ ]:
jax.vmap(callme)(init_val)

DeviceArray([[743, 461, 195, ..., 657, 948, 340],
             [813, 952, 449, ..., 590, 578, 643],
             [156, 414, 277, ..., 867, 806, 619],
             ...,
             [417,  28, 689, ..., 285,  59, 623],
             [655, 297, 661, ..., 210, 396,  60],
             [429, 344,   8, ..., 702, 235, 795]], dtype=int32)